In [1]:
! pip install numpy

In [2]:
! pip install pandas

In [3]:
import pandas as pd
import numpy as np

In [4]:
news_data = pd.read_csv("C:\\Users\\anjup\\Downloads\\abcnews-date-text.csv")
news_data.shape

(1244184, 2)

In [5]:
NUM_SAMPLES = 20000 # The number of sample to use

df = news_data.sample(NUM_SAMPLES, replace=False, random_state=42).reset_index(drop=True)

In [6]:
! pip install gensim

In [7]:
! pip install nltk


In [8]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# ----- Preprocessing -----
def preprocess(doc):
    tokens = nltk.word_tokenize(doc.lower())
    return [token for token in tokens if token.isalpha() and token not in stop_words]

texts = [preprocess(doc) for doc in df['headline_text'].dropna()]

# ----- Dictionary and BoW Corpus -----
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

# ----- TF-IDF Transformation -----
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# ----- LDA using TF-IDF corpus -----
lda_model = gensim.models.LdaModel(
    corpus=tfidf_corpus,
    id2word=dictionary,
    num_topics=7,
    random_state=42,
    passes=15,
    alpha='auto',
    per_word_topics=True
)

# Optional: Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0: 0.006*"drug" + 0.004*"deal" + 0.004*"media" + 0.004*"mayor" + 0.004*"future" + 0.004*"pay" + 0.004*"rejects" + 0.004*"told" + 0.004*"river" + 0.003*"research"
Topic 1: 0.004*"life" + 0.004*"say" + 0.004*"funds" + 0.004*"job" + 0.004*"anti" + 0.004*"war" + 0.004*"un" + 0.004*"korea" + 0.004*"aussie" + 0.004*"cuts"
Topic 2: 0.006*"found" + 0.006*"calls" + 0.006*"attack" + 0.005*"farmers" + 0.005*"charges" + 0.005*"face" + 0.004*"top" + 0.004*"tourism" + 0.004*"arrested" + 0.003*"nrl"
Topic 3: 0.009*"police" + 0.008*"man" + 0.007*"new" + 0.006*"says" + 0.005*"interview" + 0.005*"nsw" + 0.005*"court" + 0.005*"govt" + 0.005*"us" + 0.005*"council"
Topic 4: 0.007*"coast" + 0.005*"gold" + 0.005*"country" + 0.004*"review" + 0.004*"drought" + 0.004*"budget" + 0.004*"mine" + 0.004*"jobs" + 0.004*"could" + 0.003*"bid"
Topic 5: 0.008*"rural" + 0.007*"news" + 0.005*"faces" + 0.004*"afl" + 0.004*"lead" + 0.004*"bushfire" + 0.004*"flood" + 0.004*"blaze" + 0.003*"denies" + 0.003*"storm"
Topic 

In [11]:
! pip install scikit-learn


In [12]:
from gensim.models import CoherenceModel
from sklearn.metrics import silhouette_score
import numpy as np

# ---------- Coherence Score ----------
coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=texts,
                                     dictionary=dictionary,
                                     coherence='c_v')
coherence_score_lda = coherence_model_lda.get_coherence()
print(" Coherence Score:", coherence_score_lda)

 Coherence Score: 0.6199605265595644


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import numpy as np

# Step 1: Convert processed texts to TF-IDF vectors
processed_docs = [" ".join(text) for text in texts]  # Join tokens into strings
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Step 2: Assign LDA topic to each document
topic_assignments = []
for bow in [dictionary.doc2bow(text) for text in texts]:
    topic_probs = lda_model.get_document_topics(bow)
    top_topic = max(topic_probs, key=lambda x: x[1])[0]  # Most probable topic
    topic_assignments.append(top_topic)

labels = np.array(topic_assignments)

# Step 3: Compute Silhouette Score
if np.unique(labels).size > 1:
    sil_score = silhouette_score(tfidf_matrix, labels)
    print("📈 Silhouette Score (LDA + TF-IDF):", round(sil_score, 4))
else:
    print("⚠️ Not enough distinct topics for Silhouette Score.")

📈 Silhouette Score (LDA + TF-IDF): -0.0704


In [14]:
from gensim import corpora, models

# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA model
lda_model = models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=7,
    passes=10,
    iterations=150,
    alpha='symmetric',
    eta='auto',
    random_state=42,
    per_word_topics=True
)

In [15]:
# Customize this mapping based on your dataset
user_type_accounts = {
    0: 'Professional',
    1: 'soc.religion.christian',
    2: 'rec.sport.baseball',
    3: 'Mixed Type',
    4: 'Celebrity',
    5: 'talk.politics.misc'
}

In [16]:
# Get dominant topic
topic_predictions = [lda_model.get_document_topics(doc, minimum_probability=0.1) for doc in bow_corpus]

# Predict user type based on dominant topicsdf
user_type_account_predictions = []

for topic_dist in topic_predictions:
    if topic_dist:
        dominant_topic = max(topic_dist, key=lambda x: x[1])[0]
    else:
        dominant_topic = -1  # Handle empty case
    user_type = user_type_accounts.get(dominant_topic, 'Unknown')
    user_type_account_predictions.append(user_type)

# Add to dataframe
df['predicted_user_type'] = user_type_account_predictions

In [17]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['topic_label'] = label_encoder.fit_transform(df['predicted_user_type'])

# Extract the label array for training
y = df['topic_label'].values

In [18]:
import numpy as np

# Each document → topic distribution vector
corpus_bow = [dictionary.doc2bow(text) for text in texts]
topic_distributions = []

for doc_bow in corpus_bow:
    topic_dist = lda_model.get_document_topics(doc_bow, minimum_probability=0.0)
    topic_vector = [prob for _, prob in topic_dist]
    topic_distributions.append(topic_vector)

X = np.array(topic_distributions)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

print("📊 Logistic Regression Accuracy:", round(accuracy_score(y_test, y_pred_log), 4))
print("\n📝 Classification Report (Logistic Regression):\n")
print(classification_report(y_test, y_pred_log))

📊 Logistic Regression Accuracy: 0.961

📝 Classification Report (Logistic Regression):

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       558
           1       0.99      0.96      0.97       626
           2       0.99      0.95      0.97       592
           3       0.96      0.97      0.96       580
           4       0.95      0.97      0.96       570
           5       0.95      0.97      0.96       509
           6       0.93      0.96      0.94       565

    accuracy                           0.96      4000
   macro avg       0.96      0.96      0.96      4000
weighted avg       0.96      0.96      0.96      4000



In [20]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\n🌲 Random Forest Accuracy:", round(accuracy_score(y_test, y_pred_rf), 4))
print("\n📝 Classification Report (Random Forest):\n")
print(classification_report(y_test, y_pred_rf))


🌲 Random Forest Accuracy: 0.9768

📝 Classification Report (Random Forest):

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       558
           1       0.98      0.99      0.98       626
           2       0.98      0.97      0.97       592
           3       0.97      0.98      0.98       580
           4       0.99      0.98      0.99       570
           5       0.98      0.97      0.97       509
           6       0.97      0.98      0.98       565

    accuracy                           0.98      4000
   macro avg       0.98      0.98      0.98      4000
weighted avg       0.98      0.98      0.98      4000



In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Instantiate and train the Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Make predictions
y_pred_nb = nb_model.predict(X_test)

# Print accuracy and classification report
print("\n🧠 Naive Bayes Accuracy:", round(accuracy_score(y_test, y_pred_nb), 4))
print("\n📝 Classification Report (Naive Bayes):\n")
print(classification_report(y_test, y_pred_nb))


🧠 Naive Bayes Accuracy: 0.9545

📝 Classification Report (Naive Bayes):

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       558
           1       1.00      0.91      0.95       626
           2       0.97      0.95      0.96       592
           3       0.95      0.96      0.96       580
           4       0.97      0.95      0.96       570
           5       0.93      0.97      0.95       509
           6       0.91      1.00      0.95       565

    accuracy                           0.95      4000
   macro avg       0.95      0.96      0.95      4000
weighted avg       0.96      0.95      0.95      4000

